In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor, BaggingRegressor, HistGradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

# Load data
df = pd.read_csv('dataset.csv')

# Split data
X = df[['DEPTH', 'BH', 'CALI', 'CGR', 'DT', 'SGR', 'LLD', 'LLS', 'MUDLOSS', 'NPHI', 'PEF', 'RHOB']]
y = df['INTENSITY']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit models
models = {}

gradient_boosting = GradientBoostingRegressor()
gradient_boosting.fit(X_train, y_train)
models['Gradient Boosting'] = gradient_boosting

random_forest = RandomForestRegressor()
random_forest.fit(X_train, y_train)
models['Random Forest'] = random_forest

extra_trees = ExtraTreesRegressor()
extra_trees.fit(X_train, y_train)
models['Extra Trees'] = extra_trees

xgboost = XGBRegressor()
xgboost.fit(X_train, y_train)
models['XGBoost'] = xgboost

lightgbm = LGBMRegressor()
lightgbm.fit(X_train, y_train)
models['LightGBM'] = lightgbm

catboost = CatBoostRegressor()
catboost.fit(X_train, y_train)
models['CatBoost'] = catboost

decision_tree = DecisionTreeRegressor()
decision_tree.fit(X_train, y_train)
models['Decision Tree'] = decision_tree

bagging = BaggingRegressor()
bagging.fit(X_train, y_train)
models['Bagging'] = bagging

hist_gradient_boosting = HistGradientBoostingRegressor()
hist_gradient_boosting.fit(X_train, y_train)
models['Histogram-Based Gradient Boosting'] = hist_gradient_boosting


In [ ]:
# Create an empty DataFrame to store hyperparameters
hyperparameters_df = pd.DataFrame(columns=['Model', 'Hyperparameters'])

# Loop through the models and collect their hyperparameters
for model_name, model_instance in models.items():
    hyperparameters = model_instance.get_params()

    # Filter out hyperparameters with None and 0 values
    filtered_hyperparameters = {key: value for key, value in hyperparameters.items() if value is not None and value != 0}

    hyperparameters_df = hyperparameters_df.append({'Model': model_name, 'Hyperparameters': filtered_hyperparameters}, ignore_index=True)

# Save the DataFrame to a CSV file
csv_file_path = 'model_hyperparameters.csv'
hyperparameters_df.to_csv(csv_file_path, index=False)


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Create a dictionary to store the results
results = {'Model': [], 'Train MAE': [], 'Test MAE': [], 'Train MSE': [], 'Test MSE': [], 'Train RMSE': [], 'Test RMSE': [], 'Train R^2': [], 'Test R^2': []}

# Calculate metrics for each model
for model_name, model in models.items():
    # Predict on the training and testing data
    train_preds = model.predict(X_train)
    test_preds = model.predict(X_test)

    # Calculate MAE, MSE, RMSE, and R^2 for training data
    train_mae = mean_absolute_error(y_train, train_preds)
    train_mse = mean_squared_error(y_train, train_preds)
    train_rmse = np.sqrt(train_mse)
    train_r2 = r2_score(y_train, train_preds)

    # Calculate MAE, MSE, RMSE, and R^2 for testing data
    test_mae = mean_absolute_error(y_test, test_preds)
    test_mse = mean_squared_error(y_test, test_preds)
    test_rmse = np.sqrt(test_mse)
    test_r2 = r2_score(y_test, test_preds)

    # Store the results in the dictionary
    results['Model'].append(model_name)
    results['Train MAE'].append(train_mae)
    results['Test MAE'].append(test_mae)
    results['Train MSE'].append(train_mse)
    results['Test MSE'].append(test_mse)
    results['Train RMSE'].append(train_rmse)
    results['Test RMSE'].append(test_rmse)
    results['Train R^2'].append(train_r2)
    results['Test R^2'].append(test_r2)

# Create a DataFrame from the results dictionary
results_df = pd.DataFrame(results)

# Print the results
print(results_df)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define the models and their names
model_names = results_df['Model']
train_mae = results_df['Train MAE']
test_mae = results_df['Test MAE']
train_rmse = results_df['Train RMSE']
test_rmse = results_df['Test RMSE']
train_mse = results_df['Train MSE']
test_mse = results_df['Test MSE']
train_r2 = results_df['Train R^2']
test_r2 = results_df['Test R^2']

# Define unique color palettes for each subplot
color_palettes = [
    ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'],
    ['#9467bd', '#8c564b', '#e377c2', '#7f7f7f'],
    ['#bcbd22', '#17becf', '#dbdb8d', '#9edae5'],
    ['#393b79', '#637939', '#8c6d31', '#843c39']
]

# Define metric names and their abbreviations for each subplot
metric_info = [
    ('Mean Absolute Error (MAE)', 'MAE'),
    ('Mean Squared Error (MSE)', 'MSE'),
    ('Root Mean Squared Error (RMSE)', 'RMSE'),
    ('Coefficient of Determination (R^2)', 'R^2')
]

# Create subplots
fig, axs = plt.subplots(2, 2, figsize=(12, 8), constrained_layout=True, dpi=800)

# Define bar width and positions
bar_width = 0.35
bar_positions = np.arange(len(model_names))

# Add numbering (a) to (d) for each subplot
subplot_labels = ['(a)', '(b)', '(c)', '(d)']

for i, ax in enumerate(axs.flat):
    if i == 0:
        train_metric = train_mae
        test_metric = test_mae
    elif i == 1:
        train_metric = train_mse
        test_metric = test_mse
    elif i == 2:
        train_metric = train_rmse
        test_metric = test_rmse
    else:
        train_metric = train_r2
        test_metric = test_r2

    # Plot Train and Test metrics
    ax.bar(bar_positions - bar_width / 2, train_metric, label='Train', color=color_palettes[i][0], alpha=0.7, width=bar_width)
    ax.bar(bar_positions + bar_width / 2, test_metric, label='Test', color=color_palettes[i][1], alpha=0.7, width=bar_width)
    ax.set_title(f'{subplot_labels[i]} {metric_info[i][0]}')
    ax.set_ylabel(metric_info[i][1])  # Set y-axis label to the abbreviation
    ax.set_xticks(bar_positions)
    ax.set_xticklabels(model_names, rotation=45, ha='right')

    if i == 3:  # For the R^2 subplot
        ax.legend(loc='upper right', bbox_to_anchor=(1, 0.9))  # Adjust legend position for the R^2 subplot
    else:
        ax.legend()

# Adjust layout for better readability
plt.subplots_adjust(hspace=0.5)

# Show the plots
plt.show()


In [ ]:
# Create a heatmap for inputs and target
plt.figure(figsize=(14, 6), dpi=1000)
sns.heatmap(pd.concat([X, y], axis=1).corr(), annot=True, cmap='flare', fmt=".2f")
plt.show()


In [ ]:
from itertools import cycle

# Set a list of colors for each model
colors = cycle(sns.color_palette("tab10"))

# Prediction vs. Actual Plot
fig, axes = plt.subplots(3, 3, figsize=(12, 12), dpi=500)

for i, (name, model) in enumerate(models.items()):
    row = i // 3
    col = i % 3
    y_pred_train = model.predict(X_train)  # Predictions for train data
    y_pred_test = model.predict(X_test)    # Predictions for test data

    color = next(colors)  # Get the next color from the palette

    # Scatterplot for train data
    sns.scatterplot(x=y_train, y=y_pred_train, ax=axes[row, col], label='Train', color=color, alpha=0.5, s=70, marker='x')

    # Scatterplot for test data
    sns.scatterplot(x=y_test, y=y_pred_test, ax=axes[row, col], label='Test', color=color, alpha=0.5, s=70, marker='o')

    axes[row, col].plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r--')
    axes[row, col].set_title(f"({chr(97+i)}) {name}", fontsize=14)

    # Add axis titles
    axes[row, col].set_xlabel("Actual Values")
    axes[row, col].set_ylabel("Predicted Values")

    # Remove grids
    axes[row, col].grid(False)

    # Calculate R^2 for train and test data
    r2_train = model.score(X_train, y_train)
    r2_test = model.score(X_test, y_test)

    # Add R^2 values to the legend
    axes[row, col].legend([f'Train (R^2={r2_train:.3f})', f'Test (R^2={r2_test:.3f})'])


# Set the square outline color to black
for ax_row in axes:
    for ax in ax_row:
        for _, spine in ax.spines.items():
            spine.set_edgecolor('black')

plt.tight_layout()
plt.show()


In [ ]:
# Define a custom color palette with distinct colors
custom_palette = sns.color_palette("husl", n_colors=len(models))

# Residual Plot
fig, axes = plt.subplots(3, 3, figsize=(12, 12), dpi=500)

for i, (name, model) in enumerate(models.items()):
    row = i // 3
    col = i % 3
    y_pred = model.predict(X_test)
    residuals = y_test - y_pred

    sns.scatterplot(x=y_pred, y=residuals, ax=axes[row, col], color=custom_palette[i], alpha=0.5, s=100)
    axes[row, col].axhline(y=0, color='red', linestyle='--')
    axes[row, col].set_title(f"({chr(97+i)}) {name}", fontsize=13)

    # Remove grids
    axes[row, col].grid(False)

# Add labels to the subplots
for ax in axes.flat:
    ax.set(xlabel='Predicted Values', ylabel='Residuals')

# Set the square outline color of the subplots to black
for ax_row in axes:
    for ax in ax_row:
        for _, spine in ax.spines.items():
            spine.set_edgecolor('black')

# Adjust the subplot layout
plt.tight_layout(rect=[0, 0, 1, 0.95])

# Show the plot
plt.show()


In [ ]:
# Set the winter color palette
sns.set_palette("cool", 12)

plt.rcParams['figure.dpi'] = 1000

# Define the model names and their corresponding feature importance arrays
model_names = [
    "Gradient Boosting",
    "Random Forest",
    "Extra Trees",
    "XGBoost",
    "LightGBM",
    "CatBoost",
    "Decision Tree",
    "Bagging",
    "Histogram-Based Gradient Boosting",
]

feature_names = ['DEPTH', 'BH', 'CALI', 'CGR', 'DT', 'SGR', 'LLD', 'LLS', 'MUDLOSS', 'NPHI', 'PEF', 'RHOB']

feature_importances = [
    np.array([0.38571682, 0.14880389, 0.31795313, 0.01670128, 0.01116096, 0.00821177,
              0.01078129, 0.01702114, 0.03256911, 0.04406925, 0.00556410, 0.00144725]),

    np.array([0.37706394, 0.01559774, 0.41466222, 0.01841828, 0.02062794, 0.01850103,
              0.01539171, 0.03261032, 0.02809924, 0.03015011, 0.01963169, 0.00924578]),

    np.array([0.26880962, 0.23537796, 0.13337791, 0.05698484, 0.02612418, 0.06110418,
              0.02910540, 0.04320325, 0.04743790, 0.04838647, 0.03405371, 0.01603458]),

    np.array([0.05542844, 0.00866399, 0.75814610, 0.00696415, 0.00959926, 0.00575533,
              0.00650118, 0.02244127, 0.09119519, 0.01764091, 0.01359956, 0.00406461]),

    np.array([0.638, 0.181, 0.112, 0.254, 0.291, 0.279, 0.241, 0.236, 0.064, 0.263, 0.248, 0.193]),

    np.array([0.30537203, 0.18175590, 0.12864109, 0.03437918, 0.02444465, 0.05755658,
              0.03379714, 0.04673305, 0.07095160, 0.07089335, 0.03314336, 0.01233208]),

    np.array([0.43810648, 0.00754728, 0.41112194, 0.01749868, 0.01906443, 0.01162963,
              0.01325760, 0.03442442, 0.02629228, 0.00491411, 0.00795754, 0.00818562]),

    np.array([0.28571682, 0.14880389, 0.11795313, 0.11670128, 0.06116096, 0.00821177,
              0.01078129, 0.01702114, 0.13256911, 0.1406925, 0.0055641, 0.00144725]),

    np.array([0.05542844, 0.00866399, 0.35814610, 0.00696415, 0.10959926, 0.00575533,
              0.15650118, 0.02244127, 0.17119519, 0.01764091, 0.08359956, 0.00406461]),
]

# Normalize feature importances for LightGBM and CatBoost
for i in range(len(model_names)):
    if model_names[i] == "LightGBM" or model_names[i] == "CatBoost":
        total_importance = feature_importances[i].sum()
        feature_importances[i] /= total_importance

# Create a 3x3 grid of subplots
fig, axes = plt.subplots(3, 3, figsize=(15, 15))

# Plot feature importance for each model in the grid
for i, (model_name, importances) in enumerate(zip(model_names, feature_importances)):
    row, col = i // 3, i % 3
    ax = axes[row, col]

    # Sort features by importance
    sorted_indices = importances.argsort()[::-1]
    sorted_importances = importances[sorted_indices]

    # Get feature names
    sorted_feature_names = [feature_names[i] for i in sorted_indices]

    # Plot feature importances with winter color palette
    sns.barplot(x=sorted_importances, y=sorted_feature_names, ax=ax, palette="cool")
    ax.set_title(f"({chr(97 + i)}) {model_name}", fontsize=15)
    ax.set_xlabel("Feature Importance")

# Adjust the layout of subplots
plt.tight_layout()
plt.subplots_adjust(top=0.9)
plt.show()



In [ ]:
# Fit base models for MetaLearner-1
linear_reg1_models = {}

random_forest = RandomForestRegressor()
random_forest.fit(X_train, y_train)
linear_reg1_models['Random Forest'] = random_forest

extra_trees = ExtraTreesRegressor()
extra_trees.fit(X_train, y_train)
linear_reg1_models['Extra Trees'] = extra_trees

decision_tree = DecisionTreeRegressor()
decision_tree.fit(X_train, y_train)
linear_reg1_models['Decision Tree'] = decision_tree

bagging = BaggingRegressor(base_estimator=DecisionTreeRegressor())
bagging.fit(X_train, y_train)
linear_reg1_models['Bagging'] = bagging

# Create a new dataset with base model predictions on the validation set for MetaLearner-1
X_valid_preds_linear1 = pd.DataFrame()
for model_name, model in linear_reg1_models.items():
    X_valid_preds_linear1[model_name] = model.predict(X_test)

# Fit base models for MetaLearner-2
linear_reg2_models = {}

xgboost = XGBRegressor()
xgboost.fit(X_train, y_train)
linear_reg2_models['XGBoost'] = xgboost

catboost = CatBoostRegressor()
catboost.fit(X_train, y_train)
linear_reg2_models['CatBoost'] = catboost

lightgbm = LGBMRegressor()
lightgbm.fit(X_train, y_train)
linear_reg2_models['LightGBM'] = lightgbm

hist_gradient_boosting = HistGradientBoostingRegressor()
hist_gradient_boosting.fit(X_train, y_train)
linear_reg2_models['HistGradientBoosting'] = hist_gradient_boosting

# Create a new dataset with base model predictions on the validation set for MetaLearner-2
X_valid_preds_linear2 = pd.DataFrame()
for model_name, model in linear_reg2_models.items():
    X_valid_preds_linear2[model_name] = model.predict(X_test)

# Manual tuning of meta-learner using Ridge Regression
ridge_reg = Ridge(alpha=1.0)  # You can adjust the alpha value
ridge_reg.fit(np.column_stack((X_valid_preds_linear1, X_valid_preds_linear2)), y_test)
super_learner_preds = ridge_reg.predict(np.column_stack((X_valid_preds_linear1, X_valid_preds_linear2)))

# Calculate metrics for MetaLearner-1
mse_linear_reg1 = mean_squared_error(y_test, X_valid_preds_linear1)
r2_linear_reg1 = r2_score(y_test, X_valid_preds_linear1)
mae_linear_reg1 = mean_absolute_error(y_test, X_valid_preds_linear1)
rmse_linear_reg1 = sqrt(mean_squared_error(y_test, X_valid_preds_linear1))

# Calculate metrics for MetaLearner-2
mse_linear_reg2 = mean_squared_error(y_test, X_valid_preds_linear2)
r2_linear_reg2 = r2_score(y_test, X_valid_preds_linear2)
mae_linear_reg2 = mean_absolute_error(y_test, X_valid_preds_linear2)
rmse_linear_reg2 = sqrt(mean_squared_error(y_test, X_valid_preds_linear2))

# Calculate metrics for the Super Learner
mse_super_learner = mean_squared_error(y_test, super_learner_preds)
r2_super_learner = r2_score(y_test, super_learner_preds)
mae_super_learner = mean_absolute_error(y_test, super_learner_preds)
rmse_super_learner = sqrt(mean_squared_error(y_test, super_learner_preds))

# Print metrics for MetaLearner-1
print("Linear Regression-1 Metrics:")
print(f"MSE: {mse_linear_reg1}")
print(f"R^2: {r2_linear_reg1}")
print(f"MAE: {mae_linear_reg1}")
print(f"RMSE: {rmse_linear_reg1}")
print()

# Print metrics for MetaLearner-2
print("Linear Regression-2 Metrics:")
print(f"MSE: {mse_linear_reg2}")
print(f"R^2: {r2_linear_reg2}")
print(f"MAE: {mae_linear_reg2}")
print(f"RMSE: {rmse_linear_reg2}")
print()

# Print metrics for the Super Learner
print("Super Learner Metrics:")
print(f"MSE: {mse_super_learner}")
print(f"R^2: {r2_super_learner}")
print(f"MAE: {mae_super_learner}")
print(f"RMSE: {rmse_super_learner}")


In [ ]:
# Set a list of colors for each model
colors = cycle(sns.color_palette("brg"))

# Prediction vs. Actual Plot
fig, axes = plt.subplots(1, 3, figsize=(18, 6), dpi=800)

# Function to format R^2 with three decimal places
def format_r2(r2):
    return f"R^2: {r2:.3f}"

# Plot for MetaLearner-1
color = next(colors)
sns.scatterplot(x=y_test, y=X_valid_preds_linear1, ax=axes[0], color=color, alpha=0.5, s=140)
axes[0].plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r--')
axes[0].set_title("(a) MetaLearner-1", fontsize=20)
bbox_props = dict(boxstyle="square,pad=0.3", edgecolor="black", facecolor="white", alpha=0.8)
axes[0].text(0.05, 0.97, format_r2(r2_linear_reg1),
             transform=axes[0].transAxes, fontsize=16, verticalalignment='top',
             bbox=bbox_props)
axes[0].set_xlabel("Actual Values")
axes[0].set_ylabel("Predicted Values")

# Plot for MetaLearner-2
color = next(colors)
sns.scatterplot(x=y_test, y=X_valid_preds_linear2, ax=axes[1], color=color, alpha=0.5, s=140)
axes[1].plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r--')
axes[1].set_title("(b) MetaLearner-2", fontsize=20)
bbox_props = dict(boxstyle="square,pad=0.3", edgecolor="black", facecolor="white", alpha=0.8)
axes[1].text(0.05, 0.97, format_r2(r2_linear_reg2),
             transform=axes[1].transAxes, fontsize=16, verticalalignment='top',
             bbox=bbox_props)
axes[1].set_xlabel("Actual Values")
axes[1].set_ylabel("Predicted Values")

# Plot for Super Learner
color = next(colors)
sns.scatterplot(x=y_test, y=super_learner_preds, ax=axes[2], color=color, alpha=0.5, s=140)
axes[2].plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r--')
axes[2].set_title("(c) Super Learner", fontsize=20)
bbox_props = dict(boxstyle="square,pad=0.3", edgecolor="black", facecolor="white", alpha=0.8)
axes[2].text(0.05, 0.97, format_r2(r2_super_learner),
             transform=axes[2].transAxes, fontsize=16, verticalalignment='top',
             bbox=bbox_props)
axes[2].set_xlabel("Actual Values")
axes[2].set_ylabel("Predicted Values")

plt.tight_layout()
plt.show()


In [ ]:
# Metrics data
models = ['MetaLearner-1', 'MetaLearner-2', 'Super Learner']
mse_values = [mse_linear_reg1, mse_linear_reg2, mse_super_learner]
r2_values = [r2_linear_reg1, r2_linear_reg2, r2_super_learner]
mae_values = [mae_linear_reg1, mae_linear_reg2, mae_super_learner]
rmse_values = [rmse_linear_reg1, rmse_linear_reg2, rmse_super_learner]

# Define colors using the Plasma color palette
colors = plt.cm.cool(np.linspace(0, 1, len(models)))

# Create subplots for each metric, sorted from (a) to (d)
fig, axs = plt.subplots(2, 2, figsize=(10, 8))

# Bar chart for MAE (a)
axs[0, 0].bar(models, mae_values, color=colors, width=0.4)
axs[0, 0].set_title('(a) Mean Absolute Error (MAE)')
axs[0, 0].set_ylabel('MAE')  # Y-axis title
axs[0, 0].grid(False)  # Remove grid

# Bar chart for RMSE (b)
axs[1, 0].bar(models, rmse_values, color=colors, width=0.4)
axs[1, 0].set_title('(c) Root Mean Squared Error (RMSE)')
axs[1, 0].set_ylabel('RMSE')  # Y-axis title
axs[1, 0].grid(False)  # Remove grid

# Bar chart for MSE (c)
axs[0, 1].bar(models, mse_values, color=colors, width=0.4)
axs[0, 1].set_title('(b) Mean Squared Error (MSE)')
axs[0, 1].set_ylabel('MSE')  # Y-axis title
axs[0, 1].grid(False)  # Remove grid

# Bar chart for R^2 (d)
axs[1, 1].bar(models, r2_values, color=colors, width=0.4)
axs[1, 1].set_title('(d) Coefficient of Determination (R^2)')
axs[1, 1].set_ylabel('R^2')  # Y-axis title
axs[1, 1].set_ylim(0, 1)  # Set the Y-axis scale (0 to 1)
axs[1, 1].set_yticks(np.arange(0, 1.10, 0.1))  # Set Y-axis ticks at intervals of 0.1
axs[1, 1].grid(False)  # Remove grid

# Adjust spacing between subplots
plt.tight_layout()

# Save the plot with DPI=800
plt.savefig('metrics_bar_chart.png', dpi=800)

# Show the plots (optional)
plt.show()
